In [1]:
import CorrelationsModels as CM

In [2]:
import pandas as pd

# Define dates
dates = pd.date_range(start="2023-01-01", periods=8, freq="D")

# Create a simple x_matrix (monetary positions)
data = {
    "Asset A": [1000, 1020, 980, 990, 1010, 1005, 995, 1002],
    "Asset B": [500, 520, 510, 530, 525, 535, 540, 538],
    "Asset C": [200, 210, 205, 215, 220, 225, 230, 235]
}
x_matrix = pd.DataFrame(data, index=dates)
print("Input x_matrix:")
print(x_matrix)


Input x_matrix:
            Asset A  Asset B  Asset C
2023-01-01     1000      500      200
2023-01-02     1020      520      210
2023-01-03      980      510      205
2023-01-04      990      530      215
2023-01-05     1010      525      220
2023-01-06     1005      535      225
2023-01-07      995      540      230
2023-01-08     1002      538      235


In [ ]:
# Run RiskMetrics VaR
rm_results, rm_next = CM.var_riskmetrics(x_matrix, confidence_level=0.99)
print("\nRiskMetrics VaR (last 3 rows):")
print(rm_results.tail(3))
print("Next-day VaR (RiskMetrics):", round(rm_next, 2))

# Run VEC VaR
vec_results, vec_next = CM.var_vec(x_matrix, confidence_level=0.99)
print("\nVEC VaR (last 3 rows):")
print(vec_results.tail(3))
print("Next-day VaR (VEC):", round(vec_next, 2))



RiskMetrics VaR (last 3 rows):
            Portfolio Return  Portfolio Volatility        VaR  VaR Violation
2023-01-06          0.005852             36.025137  83.807000          False
2023-01-07          0.000146             34.966730  81.344777          False
2023-01-08          0.005727             34.205839  79.574681          False
Next-day VaR (RiskMetrics): 79.57

VEC VaR (last 3 rows):
            Portfolio Return  Portfolio Volatility        VaR  VaR Violation
2023-01-06          0.005852             36.013755  83.780522          False
2023-01-07          0.000146             35.522723  82.638210          False
2023-01-08          0.005727             35.192290  81.869508          False
Next-day VaR (VEC): 81.87


In [10]:
# Sample data (8 periods, 2 assets)
dates = pd.date_range(start="2023-01-01", periods=8, freq="D")
data = {
    "Asset A": [1000, 1020, 980, 990, 1010, 1005, 995, 1002],
    "Asset B": [500, 520, 510, 530, 525, 535, 540, 538]
}
x_matrix = pd.DataFrame(data, index=dates)

# Run Rolling MA VaR with window size 3
ma_results, ma_next = CM.var_movingaverage(x_matrix, confidence_level=0.99, window_size=3)
print("\nMoving Average VaR (last 3 rows):")
print(ma_results.tail(3))
print("Next-day VaR (Moving Average):", round(ma_next, 2))


Moving Average VaR (last 3 rows):
            Portfolio Return  Portfolio Volatility        VaR  VaR Violation
2023-01-06          0.003387             13.027128  30.305631          False
2023-01-07         -0.003162             10.128056  23.561381          False
2023-01-08          0.003284              5.795786  13.483015          False
Next-day VaR (Moving Average): 13.59
